In [3]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# send_keys模擬鍵盤輸入
from selenium.webdriver.common.keys import Keys

# ChromeDriver 的下載管理工具
from webdriver_manager.chrome import ChromeDriverManager

# 下載檔案的工具
import wget

# 文字修飾
from pprint import pprint,pformat

from bs4 import BeautifulSoup

import pandas as pd

'''
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
'''
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")           #最大化視窗
my_options.add_argument("--incognito")                 #開啟無痕模式
my_options.add_argument("--disable-popup-blocking")    #禁用彈出攔截
my_options.add_argument("--disable-notifications")     #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")                #設定為正體中文


# 使用 Chrome 的 WebDriver
# my_service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(
    options = my_options,
#     service = my_service
)

In [4]:
# 從文本文件中讀取景點名稱
titles = []
articles_data = []
with open('小琉球景點.txt', 'r', encoding='utf-8') as file:
    title_read = eval(file.read())
    # 印出每一個景點名稱
    for sublist in title_read:
        title_pattern = sublist[0]
        titles.append(title_pattern)
print(titles)

# 遍歷所有景點名稱，爬取維基百科上的信息
for title in titles:
    url = f'https://zh.wikipedia.org/zh-tw/{title}'
    driver.get(url)

    # 使用WebDriverWait等待頁面元素加載
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'firstHeading'))
        )
    except TimeoutException:
        print(f"頁面 {title} 加載超時")
        continue

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    title = soup.find('h1', id='firstHeading')
    content = soup.find('div', class_='mw-parser-output')

    if content and title:
        article_info = {
            'title': title.get_text().strip(),
            'content': ' '.join([p.get_text() for p in content.find_all('p') if p.get_text().strip() != ''])
        }
        articles_data.append(article_info)
    else:
        print(f"頁面 {title} 是空白頁面或不包含預期內容")

# 關閉瀏覽器
driver.quit()

# 將文章數據轉換成DataFrame，並保存到CSV文件
df = pd.DataFrame(articles_data)
df.to_csv('wikipedia_data.csv', index=False, encoding='utf-8-sig')

['美人洞', '望海亭', '山豬溝生態步道', '百年老榕樹', '烏鬼洞', '竹林生態濕地公園', '網美老木', '蛤板灣海灘', '小琉球花瓶岩', '龍蝦洞', '厚石魚澳', '杉福生態廊道', '中澳沙灘', 'Ryukyu Charming Business District｜Xiaoliuqiu Art Street', '拉美FUN+園區', '老鼠岩', '落日亭', '小琉球必吃必玩景點', '花瓶石', '白燈塔', '觀音石', '旭日亭', '貓咪石', '小琉球三民老街', '網美老石', '類火車', '山豬溝原始森林步道', '美人沙灘', '小琉球特色海龜郵筒', '潮間帶生態區', '肚仔坪潮間帶', '大寮-貝殼屋', '山豬溝海味珍特產店', '觀日公園', '小琉球', '小琉球澎坊免稅商店－觀景平台', '小琉球海龜特色郵筒', '美人洞風景區第二售票亭', '小琉球不是球', '小琉球天涯海角', '板蛤沙灘', '海龜石', '杉福生態漁港海灘休憩涼亭', '觀音亭站牌', '蟾蜍岩', '對朋友友善（Be Kind To our Friends)', '琉球海洋驛站', '肚仔坪碉堡', '琉球夕照 觀海樓台', '倩女臺', 'a點燈塔（進出港專用）', '小琉球海景渡假區/高位礁岩/天坑景觀', '小山看海藝文園區', '漁福亭', '琉球新漁港遊客中心', '小琉球猩猩石', '厚石海岸', '小琉球杉福潮間帶', '白桂淳看書', '花瓶岩步道濱海潮間帶', '琉球朝天宮熊惦夜市', '探索拉美', '琉球新船運服務中心', '八福育樂有限公司小琉球觀光海底玻璃船/小琉球觀光船/海底玻璃船/小琉球觀光景點/半潛艇/水上活動/聯營處交通船', '海子口漁港', '白沙尾漁港', '海龜墳場', 'Boracay Taiwan', '看海不是海', '小琉球靈山寺', '小琉球碧雲寺觀音佛祖', '墾丁國家公園', '小琉球夜市', '山川琉璃吊橋', '牛埔泥岩水土保持教學園區', '漁福漁港', '龍山號生態之旅', '七股鹽山', '九棚大沙漠', '關子嶺風景區']
